In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
#
import importlib
import utilities.lstm_utils as lstm_utils
import utilities.mpt_utils as mpt_utils
import utilities.variables as variables

In [22]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [23]:
df = pd.read_csv('../../../data/df_monthly_prices_complete_euro.csv', index_col='Date')
df_pct = pd.read_csv('../../../data/df_monthly_returns_complete.csv', index_col='Date')
df_static = pd.read_csv('../../../data/df_overview.csv', index_col=0)

## LSTM Model

In [24]:
# Define multivariate LSTM model
class LSTM_Multi_Model(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1, output_size=1, learning_rate=0.001, dropout=0.2): # , hidden_size=128
        super(LSTM_Multi_Model, self).__init__()
        self.hidden_size = hidden_size
        # init LSTM
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            # num_layers=num_layers,
                            batch_first=True)

        # FC layer for final prediction
        self.fc_final = nn.Linear(hidden_size, 12)

    def forward(self, ts_batch): # ts_batch (64, 1653, 10), static_data (64, 1653, 44)
        # Time-Series Data
        # Reshape dynamic data for LSTM (requires time-step as 2nd dimension)
        batch_size, num_stocks, sequence_length = ts_batch.shape[0], ts_batch.shape[1], ts_batch.shape[2]
        ts_batch_reshaped = ts_batch.view(batch_size * num_stocks, sequence_length)
        #
        ts_output_1, (hidden, cell)  = self.lstm(ts_batch_reshaped) # ts_batch_reshaped
        #
        ts_output = ts_output_1.view(batch_size, num_stocks, self.hidden_size)

        return self.fc_final(ts_output)

We use a 12 month lookback for the sequential data to predict the upcoming 12 months.

After that, based on the currently predicted time-horizon, we get the respective sub-range, 
be it 1-month, 6-month or 12-month ahead.

In [25]:
# Set sequence length (12 months)
in_seq_length = 12
out_seq_length = 12
#
out_seq_length_1m = 1
out_seq_length_6m = 6
out_seq_length_12m = 12

### LSTM Multivariate

In [26]:
df_to_evaluate = df_pct.copy()# - 1

#### Train-Test Splits

Split the data into training and testing sets

In [27]:
df_to_evaluate

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
1999-11-01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
1999-12-01,1.05,0.95,1.10,1.01,1.08,1.10,1.23,1.00,0.59,1.04,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
2000-01-01,0.91,1.02,1.00,0.99,0.92,0.98,0.77,1.00,1.07,0.61,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
2000-02-01,1.30,7.37,7.63,5.87,3.40,9.23,8.43,1.00,8.14,7.46,...,1.00,0.97,1.12,1.05,1.00,1.03,0.98,0.99,0.99,1.01
2000-03-01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,9.32,1.00,0.88,...,1.00,1.00,0.93,1.00,1.00,1.00,1.00,1.00,1.00,1.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,2.09,0.87,1.09,1.08,1.11,0.96,1.25,0.92,0.97,1.13,...,1.01,0.96,0.97,1.05,1.02,1.03,0.97,1.03,1.06,0.98
2024-06-01,1.07,0.99,1.00,0.98,1.06,0.89,0.94,0.98,1.05,0.87,...,0.96,0.99,1.00,1.03,1.06,0.95,1.01,0.93,0.95,1.00
2024-07-01,0.92,1.10,1.02,0.98,1.17,1.07,0.92,1.22,1.08,1.05,...,1.30,1.08,1.06,1.12,1.12,1.02,1.12,1.04,1.19,1.00


## Prepare static shape

In [28]:
df_static_columns = []
# Industry
df_static_columns = [s for s in df_static.columns.to_list() if "industry_" in s]
# Stock ticker
df_static_columns.append('stock_ticker_label')
#
df_static_columns.append('company_esg_score')
#
df_static_columns.append('market_capital_scale')
df_static_columns.append('trailing_pe')
df_static_columns.append('beta')
df_static_columns.append('return_on_equity')

In [29]:
df_static = df_static[df_static_columns]

In [30]:
importlib.reload(lstm_utils)
importlib.reload(variables)

# Set sequence length (e.g., 12 time points)
X_train, X_test, y_train, y_test = lstm_utils.split_train_test(df_to_evaluate, df_static, 
                                                               in_seq_length=in_seq_length, 
                                                               out_seq_length=out_seq_length, 
                                                               validation_months=(variables.TEST_YEARS_NR * 12),
                                                               )

# Check the shapes of the training and test data
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: torch.Size([263, 1332, 12])
Shape of y_train: torch.Size([263, 1332, 12])
Shape of X_test: torch.Size([12, 1332, 12])
Shape of y_test: torch.Size([12, 1332, 12])


### Model Training

In [31]:
# Model, Loss, Optimizer
model = LSTM_Multi_Model(input_size=in_seq_length, output_size=out_seq_length).to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

importlib.reload(lstm_utils)
#
model, y_train_pred, y_test_pred = lstm_utils.lstm_train_validate(model, optimizer, X_train, X_test, y_train, y_test, epochs=10)

Epoch 1/10, Loss: 0.6741, Train RMSE: 0.8430, Test RMSE: 0.4117. 
Epoch 2/10, Loss: 0.0946, Train RMSE: 0.3157, Test RMSE: 0.1477. 
Epoch 3/10, Loss: 0.0274, Train RMSE: 0.1630, Test RMSE: 0.1798. 
Epoch 4/10, Loss: 0.0219, Train RMSE: 0.1505, Test RMSE: 0.1097. 
Epoch 5/10, Loss: 0.0162, Train RMSE: 0.1280, Test RMSE: 0.1092. 
Epoch 6/10, Loss: 0.0147, Train RMSE: 0.1218, Test RMSE: 0.1077. 
Epoch 7/10, Loss: 0.0139, Train RMSE: 0.1185, Test RMSE: 0.1046. 
Epoch 8/10, Loss: 0.0138, Train RMSE: 0.1183, Test RMSE: 0.1035. 
Epoch 9/10, Loss: 0.0137, Train RMSE: 0.1175, Test RMSE: 0.1038. 
Epoch 10/10, Loss: 0.0136, Train RMSE: 0.1174, Test RMSE: 0.1034. 
Model training complete and saved.


### LSTM Multivariate - 1 Month

Get the known data (train data).
After that, get the first predicted month, or the first predicted sequence of test data

In [32]:
# 1 month
df_train = X_train[:, :, -1].clone()
y_test_pred_1m = y_test_pred[0,:,:].T[0:0] # y_test_pred[:,:,0][0] 
y_test_pred_1m

tensor([], size=(0, 1332))

In [33]:
df_forecast_1m = pd.DataFrame(df_train.clone())
df_forecast_1m = pd.concat([df_forecast_1m, pd.DataFrame(y_test_pred_1m)], ignore_index=True)
# Assign back columns and indices to make human understandable
df_forecast_1m.columns = df.columns
df_forecast_1m.index = pd.to_datetime(df_pct[(in_seq_length - 1) : len(df_forecast_1m) + (in_seq_length - 1)].index)
#
df_forecast_1m = df_forecast_1m.tail(variables.TEST_YEARS_NR * 12) 
#
df_forecast_1m.tail(3)

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-01,0.98,0.96,0.99,0.92,1.03,0.88,0.87,0.89,0.86,0.86,...,0.85,1.00,1.01,0.97,0.83,1.0,0.95,0.91,1.02,0.91
2022-07-01,1.11,1.13,0.88,0.98,0.89,1.15,1.47,1.27,1.25,1.19,...,1.14,1.00,1.08,1.12,1.31,1.0,1.13,1.09,0.94,0.95
2022-08-01,0.84,1.01,0.93,0.99,1.12,1.00,1.03,0.86,1.01,0.92,...,0.99,0.91,0.94,0.84,0.95,1.0,0.88,0.96,1.03,0.99


In [ ]:
df_forecast_1m = df_forecast_1m - 1

In [50]:
importlib.reload(lstm_utils)
importlib.reload(mpt_utils)

weights_1m, mu_1m, S_1m, allocations_1m, weights_all_1m = mpt_utils.portfolio_and_plot(df_forecast_1m, df)

Expected annual return: 49.8%
Annual volatility: 17.2%
Sharpe Ratio: 2.79
-- Allocation --
{'2395.T': 15, 'TEP.L': 1, '2767.T': 12, '3186.T': 17, '7451.T': 10, '6460.T': 12, '2685.T': 8, '5334.T': 5, '4568.T': 4, '9433.T': 5, '9987.T': 5, '3191.T': 12, '7433.T': 2, '6430.T': 4, '6417.T': 14, '7734.T': 5, '8129.T': 7, 'VOD.L': 1, '8897.T': 42, '6055.T': 6, '3402.T': 27, '8309.T': 6, 'SLB': 2, 'CAL': 5, 'ADV.DE': 4, '2379.T': 6, 'DLTR': 1, 'RELL': 6, '8804.T': 7, '3231.T': 4, 'HRB': 1, '5988.T': 5, '9434.T': 7, '9627.T': 3, '8141.T': 8, '7974.T': 1, '8219.T': 5, '8060.T': 2, 'LRN': 1, 'GBF.DE': 2, '8802.T': 5, 'CPRX': 2, 'IOT': 3, '8923.T': 4, '4732.T': 5, '9024.T': 6, '7906.T': 3, '8282.T': 6, '9869.T': 2, '6471.T': 11, '3048.T': 7, 'RGP': 3, 'BLCO': 3, 'MERC': 5, '7313.T': 4, '7867.T': 3, '4665.T': 1, '8252.T': 1, '2792.T': 3, '2874.T': 4, 'CHEF': 1, '7552.T': 1, '2154.T': 1, '3086.T': 1, '3289.T': 1}
-- Weights Percentage --
{'2395.T': 0.0584, '2685.T': 0.0255, '2767.T': 0.0456, '6430

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Actual return rate

In [51]:
importlib.reload(mpt_utils)
#
mpt_utils.get_portfolio_real_return_rate(df_pct, df_forecast_1m, weights=weights_all_1m)

Portfolio real return rate:  0.5%


#### Overview table

In [52]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(allocations_1m, mu_1m, S_1m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
2395.T,15,-0.080781,141.87%,-37.13%,-37.52%
TEP.L,1,0.002969,25.42%,-25.04%,26.40%
2767.T,12,0.037031,41.45%,56.75%,-0.47%
3186.T,17,-0.031562,182.36%,-26.44%,-22.57%
7451.T,10,0.016406,43.81%,37.33%,34.63%
...,...,...,...,...,...
CHEF,1,0.037969,6.95%,-46.81%,83.27%
7552.T,1,0.035156,1.68%,30.66%,68.61%
2154.T,1,-0.017188,21.98%,6.06%,14.74%
3086.T,1,0.047187,10.57%,23.85%,-1.41%


### LSTM Multivariate - 6 Months

Get the known data (train data)
After that, get the first 6 predicted months, or the first 6 predicted sequences of test data

In [53]:
df_train = X_train[:, :, -1].clone()
y_test_pred_6m =  y_test_pred[0,:,:].T[0:6] # y_test_pred[:,:,5][0:6]
y_test_pred_6m

tensor([[0.2076, 0.4302, 0.6152,  ..., 1.0108, 1.0110, 1.0090],
        [0.3204, 0.5253, 0.6845,  ..., 1.0104, 1.0117, 1.0083],
        [0.2218, 0.4502, 0.6263,  ..., 1.0115, 1.0135, 1.0099],
        [0.2854, 0.4939, 0.6586,  ..., 1.0121, 1.0114, 1.0097],
        [0.2784, 0.4823, 0.6477,  ..., 1.0149, 1.0171, 1.0168],
        [0.2298, 0.4462, 0.6242,  ..., 1.0112, 1.0121, 1.0102]])

In [65]:
df_forecast_6m = pd.DataFrame(df_train.clone())
df_forecast_6m = pd.concat([df_forecast_6m, pd.DataFrame(y_test_pred_6m)], ignore_index=True)
# Assign back columns and indices to make human understandable
df_forecast_6m.columns = df.columns
df_forecast_6m.index = pd.to_datetime(df_pct[(in_seq_length - 1) : len(df_forecast_6m) + (in_seq_length - 1)].index)
#
df_forecast_6m = df_forecast_6m.tail(variables.TEST_YEARS_NR * 12)
#
df_forecast_6m = df_forecast_6m - 1
df_forecast_6m.tail(3)

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-01,-0.714573,-0.506125,-0.341444,-0.207467,-0.130692,-0.080727,-0.049820,-0.029323,-0.020406,-0.010377,...,0.012994,0.010270,0.011596,0.011437,0.015287,0.013496,0.012460,0.012138,0.011398,0.009730
2023-01-01,-0.721586,-0.517744,-0.352267,-0.222002,-0.144412,-0.093396,-0.060017,-0.034806,-0.019665,-0.007940,...,0.016104,0.015714,0.014632,0.016749,0.014550,0.017673,0.016033,0.014857,0.017097,0.016756
2023-02-01,-0.770177,-0.553828,-0.375831,-0.239026,-0.149689,-0.093326,-0.062449,-0.038694,-0.022369,-0.013876,...,0.010844,0.012128,0.010403,0.008796,0.010514,0.012169,0.011137,0.011208,0.012090,0.010197


In [76]:
importlib.reload(lstm_utils)
importlib.reload(mpt_utils)

weights_6m, mu_6m, S_6m, allocations_6m, weights_all_6m = mpt_utils.portfolio_and_plot(df_forecast_6m, df, plot_threshold=0.01)

Expected annual return: 21.0%
Annual volatility: 5.7%
Sharpe Ratio: 3.33
-- Allocation --
{'TAL': 5, '7451.T': 2, 'CHEF': 3, '2379.T': 2, '2685.T': 2, '6055.T': 3, '7593.T': 17, '2670.T': 3, 'BLCO': 3, '3231.T': 1, 'AV.L': 1, 'FCN': 1, '4568.T': 1, '8129.T': 2, 'PSON.L': 1, '2784.T': 3, 'IBM': 1, 'MIDW.L': 1, 'DORM': 1, 'PLUS.L': 1, '9882.T': 3, 'VRTX': 1, '8897.T': 13, '3086.T': 4, '9869.T': 1, '5101.T': 1, '9024.T': 4, '6430.T': 1, '3289.T': 6, '9832.T': 4, '3402.T': 7, '4665.T': 1, '9413.T': 2, '8014.T': 1, '9434.T': 3, '9076.T': 3, '8309.T': 2, '9433.T': 1, '7606.T': 3, '8141.T': 4, 'SYY': 1, 'HURN': 1, '3116.T': 2, '3421.T': 4, '4745.T': 13, '9842.T': 4, '3738.T': 3, '7976.T': 3, '8803.T': 1, '9470.T': 7, '9409.T': 4, '4503.T': 3, '7518.T': 2, '8173.T': 2, '3191.T': 3, '8008.T': 3, 'FINV': 7, '8802.T': 2, '8057.T': 1, '7250.T': 3, '8804.T': 2, '3003.T': 4, '3591.T': 1, '3048.T': 4, '8130.T': 1, '2692.T': 1, '6745.T': 2, '2874.T': 4, 'FRPH': 1, '8282.T': 3, '8150.T': 2, '4544.T': 2

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Actual return rate

In [77]:
importlib.reload(mpt_utils)
#
mpt_utils.get_portfolio_real_return_rate(df_pct, df_forecast_6m, weights=weights_all_6m)

Portfolio real return rate:  0.18%


#### Overview table

In [78]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(allocations_6m, mu_6m, S_6m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
TAL,5,0.040479,26.62%,84.94%,-17.19%
7451.T,2,0.030359,37.20%,37.33%,34.63%
CHEF,3,0.010120,34.73%,-46.81%,83.27%
2379.T,2,-0.006407,71.99%,-26.08%,-19.13%
2685.T,2,-0.015689,5.44%,61.48%,22.88%
...,...,...,...,...,...
7868.T,2,0.040359,5.09%,95.94%,-16.15%
4301.T,1,-0.000838,1.06%,-17.50%,-7.97%
ACTG,2,-0.026168,17.30%,-9.92%,29.19%
7613.T,1,0.000419,14.88%,26.43%,-27.69%


### LSTM Multivariate - 12 Months

Get the known data (train data)
After that, get the first 12 predicted months, or the first 12 predicted sequences of test data

In [79]:
df_train = X_train[:, :, -1].clone()
y_test_pred_12m = y_test_pred[0,:,:].T[0:12]
y_test_pred_12m

tensor([[0.2076, 0.4302, 0.6152,  ..., 1.0108, 1.0110, 1.0090],
        [0.3204, 0.5253, 0.6845,  ..., 1.0104, 1.0117, 1.0083],
        [0.2218, 0.4502, 0.6263,  ..., 1.0115, 1.0135, 1.0099],
        ...,
        [0.1560, 0.3965, 0.5837,  ..., 1.0101, 1.0106, 1.0074],
        [0.3093, 0.5215, 0.6844,  ..., 1.0138, 1.0146, 1.0137],
        [0.3150, 0.5139, 0.6719,  ..., 1.0094, 1.0106, 1.0067]])

In [86]:
df_forecast_12m = pd.DataFrame(df_train.clone())
df_forecast_12m = pd.concat([df_forecast_12m, pd.DataFrame(y_test_pred_12m)], ignore_index=True)
# Assign back columns and indices to make human understandable
df_forecast_12m.columns = df.columns
df_forecast_12m.index = pd.to_datetime(df_pct[(in_seq_length - 1) : len(df_forecast_12m) + (in_seq_length - 1)].index)
#
df_forecast_12m = df_forecast_12m - 1
df_forecast_12m.tail(3)

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-01,-0.844049,-0.603536,-0.416329,-0.261809,-0.170922,-0.110500,-0.073046,-0.045093,-0.028594,-0.018424,...,0.012764,0.011060,0.007476,0.006396,0.012362,0.011413,0.011101,0.010142,0.010612,0.007378
2023-07-01,-0.690721,-0.478547,-0.315606,-0.189298,-0.118985,-0.073514,-0.048160,-0.024656,-0.017022,-0.006688,...,0.014051,0.013972,0.011675,0.012753,0.015292,0.015073,0.014234,0.013753,0.014631,0.013670
2023-08-01,-0.684967,-0.486062,-0.328137,-0.207250,-0.133090,-0.088168,-0.060182,-0.031226,-0.018818,-0.009726,...,0.010039,0.011314,0.005690,0.008250,0.009653,0.010735,0.011522,0.009388,0.010627,0.006743


In [87]:
importlib.reload(lstm_utils)
importlib.reload(mpt_utils)

weights_12m, mu_12m, S_12m, allocations_12m, weights_all_12m = mpt_utils.portfolio_and_plot(df_forecast_12m, df)

Expected annual return: 17.7%
Annual volatility: 1.2%
Sharpe Ratio: 13.17
-- Allocation --
{'NBPE.L': 1, '9107.T': 38, 'TWI': 37, '9104.T': 17, 'BELFB': 8, 'MEGP.L': 3, 'LAUR': 32, 'PFGC': 8, '4293.T': 154, 'JKHY': 2, '7747.T': 19, 'BFAM': 4, 'CI': 1, 'DRQ': 12, 'SVC': 43, '9101.T': 9, 'GBF.DE': 6, 'GEN': 13, 'PRDO': 11, 'MERC': 26, 'APLE': 11, '6750.T': 13, 'ECV.DE': 10, 'CRAI': 1, 'AXP': 1, 'CGNX': 3, '3341.T': 9, '7575.T': 11, 'AL': 2, '2389.T': 8, '6383.T': 2, '7483.T': 1, 'INN': 2}
-- Weights Percentage --
{'NBPE.L': 0.0573, '4293.T': 0.0522, 'MERC': 0.0241, 'PRDO': 0.0246, '9107.T': 0.0573, 'TWI': 0.0573, '7747.T': 0.0387, 'GEN': 0.0254, '9104.T': 0.0573, 'BELFB': 0.0573, 'MEGP.L': 0.0573, 'CI': 0.0339, 'JKHY': 0.0422, 'GBF.DE': 0.0276, '9101.T': 0.03, 'DRQ': 0.031, 'SVC': 0.0302, 'BFAM': 0.0377, 'PFGC': 0.0551, 'LAUR': 0.0573, 'Other(13)': 0.1463}


/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Actual return rate

In [88]:
df_pct

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
1999-11-01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
1999-12-01,1.05,0.95,1.10,1.01,1.08,1.10,1.23,1.00,0.59,1.04,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
2000-01-01,0.91,1.02,1.00,0.99,0.92,0.98,0.77,1.00,1.07,0.61,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
2000-02-01,1.30,7.37,7.63,5.87,3.40,9.23,8.43,1.00,8.14,7.46,...,1.00,0.97,1.12,1.05,1.00,1.03,0.98,0.99,0.99,1.01
2000-03-01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,9.32,1.00,0.88,...,1.00,1.00,0.93,1.00,1.00,1.00,1.00,1.00,1.00,1.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,2.09,0.87,1.09,1.08,1.11,0.96,1.25,0.92,0.97,1.13,...,1.01,0.96,0.97,1.05,1.02,1.03,0.97,1.03,1.06,0.98
2024-06-01,1.07,0.99,1.00,0.98,1.06,0.89,0.94,0.98,1.05,0.87,...,0.96,0.99,1.00,1.03,1.06,0.95,1.01,0.93,0.95,1.00
2024-07-01,0.92,1.10,1.02,0.98,1.17,1.07,0.92,1.22,1.08,1.05,...,1.30,1.08,1.06,1.12,1.12,1.02,1.12,1.04,1.19,1.00


In [92]:
df_forecast_12m = df_forecast_12m +1

In [93]:
importlib.reload(mpt_utils)
#
mpt_utils.get_portfolio_real_return_rate(df_pct, df_forecast_12m, weights=weights_all_12m)

Portfolio real return rate:  0.23%


#### Overview Table

In [94]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(allocations_12m, mu_12m, S_12m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
NBPE.L,1,0.0,18.61%,-0.25%,-28.42%
9107.T,38,0.0,13.63%,154.35%,17.39%
TWI,37,0.0,14.21%,-24.07%,-40.87%
9104.T,17,0.0,16.64%,45.45%,22.66%
BELFB,8,0.0,19.63%,69.99%,37.91%
MEGP.L,3,0.0,23.92%,59.61%,26.35%
LAUR,32,0.0,32.57%,17.84%,9.13%
PFGC,8,0.0,10.66%,11.01%,21.68%
4293.T,154,0.0,13.36%,-3.12%,4.21%
JKHY,2,0.0,20.95%,-27.16%,16.90%
